In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from urllib.parse import urlencode
import requests
from requests import Response
import datetime as dt
from datetime import time, date, timedelta
from dateutil.relativedelta import relativedelta
import itertools
from math import ceil
import scipy.stats as stats

In [8]:
from ta.momentum import WilliamsRIndicator
def WillR(window_para, highband, to_csv):
    lowband = -100 - highband
    result = pd.read_csv('raw-weekly-pricevolume.csv')
    codes = result['code'].unique().tolist()
    for code in codes:
        close_weekly = result[result['code']==code]['close_weekly']
        high_weekly = result[result['code']==code]['high_weekly']
        low_weekly = result[result['code']==code]['low_weekly']
        result.loc[code == result['code'].shift(window_para),'wr'] = WilliamsRIndicator(high = high_weekly, low = low_weekly, close=close_weekly, lbp=window_para).williams_r()
    result.loc[result['code'] == result['code'].shift(1),'return'] = result['close_weekly']/result['close_weekly'].shift(1) - 1
    result = result.dropna()
    result.loc[(result['code'] == result['code'].shift(1))&(result['wr']<lowband),'signal'] = 1
    result.loc[(result['code'] == result['code'].shift(1))&(result['wr']>highband),'signal'] = 0
    result.loc[result['code'] != result['code'].shift(1),'signal'] = 0
    result = result.ffill()
    
    result.loc[result['code'] == result['code'].shift(1),'daily_return'] = result['signal'].shift(1)*result['return']
    result.loc[(result['code'] == result['code'].shift(1))&(result['signal']==1)&(result['signal'].shift(1)==0),'daily_return']=result['daily_return']-0.005
    
    dailyprofit = pd.pivot(result, index = 'date', columns = 'code', values = 'daily_return')
    dailyprofit = dailyprofit.replace(0, np.nan)
    dailyprofit.reset_index(inplace = True)
    day_return = dailyprofit.copy()
    day_return['count'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].count(axis = 1) #exclude date column
    day_return.loc[day_return['count']<5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)*(day_return['count']/5)
    day_return.loc[day_return['count']==0,'profit'] = 0
    day_return.loc[day_return['count']>=5,'profit'] = day_return.iloc[:,1:len(dailyprofit.columns.to_list())].mean(axis = 1,skipna=True)
    
    profit = (day_return['profit'] +1).cumprod()
    day_return['cumulative'] = profit
    if to_csv:
        day_return.to_csv('checkrsi.csv')
#     cul_arr = day_return['cumulative'].to_list()
#     dd_arr = []
#     maxcul = 0
#     for x in day_return.cumulative.dropna().tolist():
#         if x >= maxcul:
#             maxcul = x
#             dd_arr.append(0)
#         else:
#             dd_arr.append((maxcul-x)/maxcul)
#     print(f'MDD {max(dd_arr)}')
    return float(profit[-1:]), window_para, highband

In [3]:
optimize_result = []
for window_para in range(2,30, 2):
    for highband in range(-50,-5, 2):
        x=WillR(2,highband, to_csv = False)
        optimize_result.append(x)    
optimize_result = pd.DataFrame(optimize_result)

In [6]:
optimize_result.sort_values(by=0, ascending =False ).head(50)

,0,1,2
19,4.676535,2,-12
18,4.674314,2,-14
94,4.585689,10,-46
17,4.560336,2,-16
95,4.496155,10,-44
22,4.461950,2,-6
96,4.459111,10,-42
20,4.428954,2,-10
21,4.428772,2,-8
93,4.415677,10,-48


In [9]:
WillR(2,-12, to_csv = False)

NameError: name 'dd_arr' is not defined